In [1]:
# Importando bibliotecas
from functions import *
import pandas as pd
import locale
from pathlib import Path
from datetime import datetime
import duckdb
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import warnings
import logging
import polars as pl
import shutil


logging.basicConfig(level=logging.WARNING, format='%(message)s')

warnings.filterwarnings("ignore")

timer = Temporizador()
timer.iniciar()

locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.expand_frame_repr', False)

# Detecta se o script está sendo executado de um .py ou de um notebook
try:
    caminho_base = Path(__file__).resolve().parent
except NameError:
    # __file__ não existe em Jupyter ou ambiente interativo
    caminho_base = Path.cwd()

pasta_input_parquet = caminho_base.parent / '01_INPUT_PIPELINE/01_BD_PARQUET'
arquivo_input_regras_negocio = caminho_base.parent / '01_INPUT_PIPELINE/02_REGRAS_NEGOCIO/VIQUA_REGRAS.xlsm'
pasta_staging_parquet = caminho_base.parent / '02_STAGING_PARQUET' # Armazena arquivos parquet com tratamentos, aplicações de regras, depara, etc
pasta_input_painel = caminho_base.parent / '03_INPUT_PAINEL' # Armazena arquivos que serão consumidos no painel de S&OP para os gerentes
pasta_painel = caminho_base.parent / '05_PAINEL'

# Eliminar arquivos das pastas de 02_STAGING_PARQUET e 03_INPUT_PAINEL que serão regenerados
pastas_para_limpar = [
    pasta_staging_parquet,
    pasta_input_painel,
]

for pasta in pastas_para_limpar:
    if pasta.exists() and pasta.is_dir():
        for item in pasta.iterdir():
            if item.is_file() or item.is_symlink():
                item.unlink()
            elif item.is_dir():
                shutil.rmtree(item)

print("✅ Mapeamento de pastas concluído com sucesso!")

✅ Mapeamento de pastas concluído com sucesso!


In [2]:
caminho_fato_vend = (pasta_input_parquet / "Fato_Vendas_Viqua.parquet").as_posix()
caminho_dim_prod = (pasta_input_parquet / "Dim_Produtos_Viqua.parquet").as_posix()
caminho_dim_cli  = (pasta_input_parquet / "Dim_Clientes_Viqua.parquet").as_posix()
caminho_dim_repr = (pasta_input_parquet / "Dim_Representante_Viqua.parquet").as_posix()

df_vendas_viqua = duckdb.query(f"""
    WITH
    fato AS (
        SELECT
            Chv_Produto,
            Chv_Cliente,
            STRFTIME(DATE_TRUNC('month', Dat_Entrega), '%Y%m') AS PERIODO,
            TRY_CAST(Qtd_Venda AS DOUBLE) AS QTD
        FROM read_parquet('{caminho_fato_vend}')
        WHERE Dat_Entrega IS NOT NULL
          AND Dat_Entrega >= DATE '2022-01-01'
          
    ),
    prod AS (
        SELECT
            Chv_Produto,
            TRIM(Cod_Produto) AS COD_PROD,
            UPPER(TRIM(Des_Produto)) AS DESC_PROD,
            UPPER(TRIM(Des_Grupo)) AS GRUPO_PROD,
            Cod_Familia,
            UPPER(TRIM(Des_Familia)) AS DESC_FAMILIA
        FROM read_parquet('{caminho_dim_prod}')
    ),
    cli AS (
        SELECT
            Chv_Cliente,
            TRIM(Cod_Cliente) AS COD_CLIENTE,
            UPPER(TRIM(Nom_Cliente)) AS NOM_CLIENTE,
            Chv_Representante_Cliente
        FROM read_parquet('{caminho_dim_cli}')
    ),
    repr AS (
        SELECT
            Chv_Representante,
            UPPER(TRIM(Nom_Regional)) AS REGIONAL
        FROM read_parquet('{caminho_dim_repr}')
    )
    SELECT
        p.COD_PROD,
        p.DESC_PROD,
        CAST(p.Cod_Familia AS VARCHAR) || ' - ' || p.DESC_FAMILIA AS FAMILIA,
        p.GRUPO_PROD,
        c.COD_CLIENTE,
        c.NOM_CLIENTE,
        r.REGIONAL,
        SUM(f.QTD) AS QTD,
        f.PERIODO
    FROM fato f
    LEFT JOIN prod p ON f.Chv_Produto = p.Chv_Produto
    LEFT JOIN cli c ON f.Chv_Cliente = c.Chv_Cliente
    LEFT JOIN repr r ON c.Chv_Representante_Cliente = r.Chv_Representante
    GROUP BY
        p.COD_PROD,
        p.DESC_PROD,
        p.Cod_Familia,
        p.DESC_FAMILIA,
        p.GRUPO_PROD,
        c.COD_CLIENTE,
        c.NOM_CLIENTE,
        r.REGIONAL,
        f.PERIODO
""").df()

# Eliminar zeros de vendas e vazios ou nulos
df_vendas_viqua = df_vendas_viqua[(df_vendas_viqua["QTD"] > 0) & (df_vendas_viqua["COD_CLIENTE"].notna()) & (df_vendas_viqua["COD_CLIENTE"] != 'null')]

# salva em parquet
df_vendas_viqua.to_parquet(pasta_staging_parquet / "df_vendas_viqua.parquet", index=False)

print("✅ Importação e tratamento de dados Historicos e concluídos com sucesso!")

✅ Importação e tratamento de dados Historicos e concluídos com sucesso!


In [5]:
# FIXME APAGAR ESSA CELULA APÓS VALIDAÇÃO ANNA
# Agrupar dados QTD, por COD_PROD, FAMILIA, GRUPO_PROD, PERIODO
df_vendas_viqua_validacao_Anna = df_vendas_viqua.groupby(
    ['COD_PROD', 'FAMILIA', 'GRUPO_PROD', 'PERIODO'],
    as_index=False
).agg({'QTD': 'sum'})


# # Gerar Excel para Anna
caminho_excel_saida = pasta_painel / f"VALIDACAO_ANNA_VIQUA_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df_vendas_viqua_validacao_Anna.to_excel(caminho_excel_saida, index=False)
df_vendas_viqua_validacao_Anna

,COD_PROD,FAMILIA,GRUPO_PROD,PERIODO,QTD
0,101725,85 - MOLDES,MAQUINAS E EQUIPAMENTOS,202201,10.00
1,101725,85 - MOLDES,MAQUINAS E EQUIPAMENTOS,202208,12.00
2,101725,85 - MOLDES,MAQUINAS E EQUIPAMENTOS,202209,10.00
3,101725,85 - MOLDES,MAQUINAS E EQUIPAMENTOS,202301,4.00
4,101725,85 - MOLDES,MAQUINAS E EQUIPAMENTOS,202303,10.00
...,...,...,...,...,...
34138,EXP11CO,33 - EXPOSITOR,OUTROS,202508,23.00
34139,EXP11CO,33 - EXPOSITOR,OUTROS,202509,32.00
34140,EXP11CO,33 - EXPOSITOR,OUTROS,202510,37.00
34141,EXP11CO,33 - EXPOSITOR,OUTROS,202511,24.00


In [3]:
# Carregar dados arquivo VIQUA_REGRAS
caminho_arquivo = arquivo_input_regras_negocio

# Carrregar os produtos a eliminar
guia_excel = 'PRODUTOS_ELIMINAR'
df_produtos_eliminar = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_produtos_eliminar['COD_PROD'] = df_produtos_eliminar['COD_PROD'].astype(str)
df_produtos_eliminar = df_produtos_eliminar.drop_duplicates(subset=['COD_PROD'])

# Carrregar Regionais Gestor para DEPARA de Regionais
guia_excel = 'REGIONAIS_GESTOR'
df_regionais_gestor = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_regionais_gestor = df_regionais_gestor.drop_duplicates(subset=['REGIONAL', 'REGIONAL_GESTOR'])

# Carrregar os Clientes que devem ter processo de Demanda
guia_excel = 'CLIENTES_DEMANDA'
df_clientes_demanda = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_clientes_demanda['Cod_Grupo_Cliente'] = df_clientes_demanda['Cod_Grupo_Cliente'].astype(str)
df_clientes_demanda = df_clientes_demanda.drop_duplicates(subset=['Cod_Grupo_Cliente'])

# Carregar demanda de novos produtos
guia_excel = 'PRODUTOS_LANCAMENTOS'
df_produtos_lancamento = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
# 🚨 VALIDAR SE EXISTEM DADOS
if df_produtos_lancamento.empty:
    raise ValueError(
        "❌ ERRO: Nenhuma informação foi encontrada na aba PRODUTOS_LANCAMENTOS.\n"
        "➡️ Verifique se a planilha possui dados válidos antes de executar o pipeline."
    )
df_produtos_lancamento['JANELA LANÇAMENTO'] = df_produtos_lancamento['JANELA LANÇAMENTO'].astype(str).str.strip()
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['JANELA LANÇAMENTO'] != ''].reset_index(drop=True)
df_produtos_lancamento.rename(columns={'COD': 'COD_PROD'}, inplace=True)
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['COD_PROD'].notna()].reset_index(drop=True)
df_produtos_lancamento['COD_PROD'] = df_produtos_lancamento['COD_PROD'].astype(str)

# Identifica colunas com datas válidas
col_datas = []
for col in df_produtos_lancamento.columns:
    try:
        pd.to_datetime(col, dayfirst=True, errors='raise')
        col_datas.append(col)
    except (ValueError, TypeError):
        continue

colunas_validas = ['COD_PROD'] + \
                    [col for col in df_produtos_lancamento.columns if 'CD:' in str(col)] + \
                    col_datas
df_produtos_lancamento = df_produtos_lancamento[[col for col in colunas_validas if col in df_produtos_lancamento.columns]]

# Transforma datas em linhas
df_produtos_lancamento = df_produtos_lancamento.melt(
    id_vars=[col for col in df_produtos_lancamento.columns if col not in col_datas],
    value_vars=col_datas,
    var_name='PERIODO',
    value_name='VALOR'
)
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['VALOR'].notna()].reset_index(drop=True)

# Multiplica colunas CD pelo VALOR
colunas_cd = [col for col in df_produtos_lancamento.columns if 'CD:' in str(col)]
for col in colunas_cd:
    df_produtos_lancamento[col] = df_produtos_lancamento[col] * df_produtos_lancamento['VALOR']
df_produtos_lancamento.drop(columns=['VALOR'], inplace=True)

# Transforma colunas CD em linhas
df_produtos_lancamento = df_produtos_lancamento.melt(
    id_vars=[col for col in df_produtos_lancamento.columns if col not in colunas_cd],
    value_vars=colunas_cd,
    var_name='CD',
    value_name='QTD'
)

# Carrega aba de DE-PARA
guia_excel = 'DE_PARA_CD'
df_depara = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_produtos_lancamento = df_produtos_lancamento.merge(df_depara[['DE', 'PARA']], how='left', left_on='CD', right_on='DE')
df_produtos_lancamento.drop(columns=['DE', 'CD'], inplace=True)
df_produtos_lancamento.insert(2, 'PARA', df_produtos_lancamento.pop('PARA'))
df_produtos_lancamento.rename(columns={'PARA': 'CD'}, inplace=True)

# Agrupa e limpa
df_produtos_lancamento = df_produtos_lancamento.groupby(['COD_PROD', 'CD', 'PERIODO'], as_index=False)['QTD'].sum()
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['QTD'].notna() & (df_produtos_lancamento['QTD'] != 0)].reset_index(drop=True)
df_produtos_lancamento['CD'] = df_produtos_lancamento['CD'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()

# Criar lista de produtos para eliminar, somente com  df_produtos_eliminar
# df_produtos_lancamento vamos prever devido ao problema apontado pela Anna, onde a previsão de lançamentos pode não cobrir lacunas dos períodos de S&OP
lista_produtos_eliminar = list(set(df_produtos_eliminar['COD_PROD'].tolist()))
                               
# Eliminar linhas do df_vendas_viqua que tenham COD_PROD presente na lista
df_vendas_viqua = df_vendas_viqua[~df_vendas_viqua['COD_PROD'].isin(lista_produtos_eliminar)].reset_index(drop=True)

print("✅ Importação e tratamento de dados do arquivo VIQUA_REGRAS, concluídos com sucesso!")

✅ Importação e tratamento de dados do arquivo VIQUA_REGRAS, concluídos com sucesso!


In [ ]:
print("🔄 Iniciando processo de previsão estatística...")

# ============================================================
# LIMPEZA SUAVE DE OUTLIERS
# ============================================================

def smooth_outliers(series, z=2.5):
    if series.std() == 0:
        return series
    mean = series.mean()
    std = series.std()
    upper = mean + z * std
    lower = max(0, mean - z * std)
    return series.clip(lower, upper)

# ============================================================
# CROSTON – demanda intermitente
# ============================================================

def croston(ts, h=6):
    ts = np.array(ts)
    if len(ts) == 0 or ts.sum() == 0:
        return np.zeros(h)

    demand = ts[ts > 0]
    intervals = np.diff(np.where(ts > 0)[0])
    mean_interval = intervals.mean() if len(intervals) > 0 else 1

    q = demand.mean()
    p = mean_interval

    fc = (q / p) * np.ones(h)
    return np.maximum(fc, 0)  # <<< TRAVA NEGATIVO

# ============================================================
# MODELO HÍBRIDO
# ============================================================

def forecast_sku(df_sku, horizon=6):
    df_sku = df_sku.sort_values("PERIODO")  # PERIODO já está AAAAMM
    y = df_sku["QTD"].values

    # histórico curto
    if len(y) < 3:
        fc = np.ones(horizon) * np.mean(y)
        return np.maximum(fc, 0)  # <<< TRAVA NEGATIVO

    zero_rate = (y == 0).mean()
    if zero_rate >= 0.40:
        return croston(y, horizon)  # já retorna >= 0

    try:
        model = ExponentialSmoothing(y, trend="add", seasonal=None)
        fit = model.fit(optimized=True)
        fc = fit.forecast(horizon)
        return np.maximum(fc, 0)  # <<< TRAVA NEGATIVO
    except:
        fc = np.ones(horizon) * np.mean(y)
        return np.maximum(fc, 0)  # <<< TRAVA NEGATIVO

# ============================================================
# PIPELINE COMPLETO – Consumindo df_vendas_viqua direto
# ============================================================

def gerar_forecast_viqua_df(df_vendas_viqua):
    print("🔄 Usando PERIODO no formato AAAAMM...")
    df = df_vendas_viqua.copy()

    df_group = (
        df.groupby(["COD_PROD", "PERIODO"])
          .agg(QTD=("QTD", "sum"))
          .reset_index()
    )

    df_group["QTD"] = df_group.groupby("COD_PROD")["QTD"].transform(smooth_outliers)

    print("🔄 Criando calendário futuro (Fev/26 → Jul/26)...")
    future_periods = ["202602", "202603", "202604", "202605", "202606", "202607"]

    registros = []

    print("🔄 Gerando previsão por SKU...")
    for sku, df_sku in df_group.groupby("COD_PROD"):
        fc = forecast_sku(df_sku, horizon=6)
        for per, val in zip(future_periods, fc):
            registros.append([sku, per, float(val)])

    df_forecast = pd.DataFrame(registros, columns=["COD_PROD", "PERIODO", "FORECAST"])

    return df_forecast

# ============================================================
# EXECUTAR
# ============================================================
df_forecast_produto_periodo = gerar_forecast_viqua_df(df_vendas_viqua)

🔄 Iniciando processo de previsão estatística...
🔄 Usando PERIODO já no formato AAAAMM...
🔄 Criando calendário futuro (Fev/26 → Jul/26)...
🔄 Gerando previsão por SKU...


In [ ]:
# DESAGREGAÇÃO ESTATISTICA

# Preparar histórico
df_hist = df_vendas_viqua.copy()
df_hist["PERIODO"] = df_hist["PERIODO"].astype(str)

# Calcular participação histórica (chave completa: PRODUTO + REGIONAL + CLIENTE)
df_part = (
    df_hist.groupby(["COD_PROD", "REGIONAL", "COD_CLIENTE"])
           .agg(QTD_HIST=("QTD", "sum"))
           .reset_index()
)

df_total_prod = (
    df_part.groupby("COD_PROD")["QTD_HIST"]
           .sum()
           .reset_index()
           .rename(columns={"QTD_HIST": "TOTAL_PROD"})
)

df_part = df_part.merge(df_total_prod, on="COD_PROD", how="left")
df_part["PARTICIPACAO"] = df_part["QTD_HIST"] / df_part["TOTAL_PROD"]
df_part["PARTICIPACAO"] = df_part["PARTICIPACAO"].fillna(0)

# Expandir forecast em uma única passada
df_exp = df_forecast_produto_periodo.merge(df_part, on="COD_PROD", how="left")
df_exp["FORECAST_DISTRIB"] = df_exp["FORECAST"] * df_exp["PARTICIPACAO"]

# Trazer atributos do produto e cliente (sem duplicação)
df_dim = (
    df_hist[
        [
            "COD_PROD", "DESC_PROD", "FAMILIA", "GRUPO_PROD",
            "COD_CLIENTE", "NOM_CLIENTE", "REGIONAL"
        ]
    ]
    .drop_duplicates(subset=["COD_PROD", "COD_CLIENTE", "REGIONAL"])
)

df_forecast_viqua = df_exp.merge(
    df_dim,
    on=["COD_PROD", "COD_CLIENTE", "REGIONAL"],
    how="left"
)

# Tabela final no formato padrão
df_forecast_viqua = df_forecast_viqua[
    [
        "COD_PROD", "DESC_PROD", "FAMILIA", "GRUPO_PROD", "COD_CLIENTE",
        "NOM_CLIENTE", "REGIONAL","FORECAST_DISTRIB", "PERIODO"
    ]
].rename(columns={"FORECAST_DISTRIB": "QTD"})

# Gerar arquivo Parquet
nome_df = "df_forecast_viqua"
print(f"💾 Salvando {nome_df} como Parquet...")
pl.from_pandas(df_forecast_viqua).write_parquet(pasta_staging_parquet / "df_forecast_viqua.parquet")

💾 Salvando df_forecast_viqua como Parquet...


In [ ]:
# Adicionar REGIONAL_GESTOR nos arquivos df_forecast_venda_viqua e df_vendas_viqua
df_forecast_venda_viqua = df_forecast_viqua.merge(
    df_regionais_gestor,
    on="REGIONAL",
    how="left",
    suffixes=('', '_excluir')
)

# Excluir colunas com sufixo _excluir
df_forecast_venda_viqua = df_forecast_venda_viqua[[col for col in df_forecast_venda_viqua.columns if not col.endswith('_excluir')]]

df_vendas_viqua = df_vendas_viqua.merge(
    df_regionais_gestor,
    on="REGIONAL",
    how="left",
    suffixes=('', '_excluir')
)

# Excluir colunas com sufixo _excluir
df_vendas_viqua = df_vendas_viqua[[col for col in df_vendas_viqua.columns if not col.endswith('_excluir')]]

# Unificar df_vendas_viqua e df_forecast_viqua.parquet em um único arquivo Parquet, para carregar no POWER BI
df_forecast_venda_viqua = pd.concat([df_vendas_viqua, df_forecast_viqua], ignore_index=True)

# Gerar arquivo Parquet
nome_df = "df_forecast_venda_viqua"
print(f"💾 Salvando {nome_df} como Parquet...")
df_forecast_venda_viqua.to_parquet(pasta_staging_parquet / "df_forecast_venda_viqua.parquet")

print("✅ Desagregação concluída com sucesso!")

💾 Salvando df_forecast_venda_viqua como Parquet...
✅ Desagregação concluída com sucesso!


In [105]:
# Listar Regional e Regional_Gestor com base na df_forecast_venda_viqua
df_avalia_regional_gestor_nan = df_forecast_venda_viqua[['REGIONAL', 'REGIONAL_GESTOR']].drop_duplicates().reset_index(drop=True)
df_avalia_regional_gestor_nan

,REGIONAL,REGIONAL_GESTOR
0,SANTA CATARINA 2,NaN
1,PARANA,SUL
2,RJ ES,NaN
3,IRRIGACAO 2,IRRIGACAO
4,TELEVENDAS,TELEVENDAS
5,CONSTRUTORA 1,CONSTRUTORA
6,NORDESTE 2,NORDESTE
7,IRRIGACAO 1,IRRIGACAO
8,None,NaN
9,SPC 1,NaN


In [ ]:
'''
PONTOS PENDENTES:
- Ver com Anna Regionais Gestor ficaram NAN
- Ver as alterações de nomes de pastas do diretório do projeto
- Considerar para definição dos períodos, a tabela nova nas regras de negócio
'''

'\nPONTOS PENDENTES:\n- Ver com Anna Regionais Gestor ficaram NAN\n'

In [107]:
timer.finalizar()
print("🎯 Processo concluído com sucesso!")


⏱️ Tempo total de processamento: 0 min 24.9 s
🎯 Processo concluído com sucesso!


In [ ]:
# Comando criar Requirements.txt
# pip freeze > ".\00_SCRIPTS\requirements.txt"